In [2]:
import pandas as pd
import numpy as np
df_faces = pd.read_csv('UIdata/faces.csv')
df_attr = pd.read_excel('UIdata/alphapolitica_faces_attributes_coords.xlsx')

In [263]:
'''Generate folium plot'''

loc_dict = dict(zip(df_attr.image_id, df_attr.coords))
per_dict = dict(zip(df_attr.image_id, df_attr.Name))
constituency_dict = dict(zip(df_attr.image_id, df_attr.constituency))
party_dict = dict(zip(df_attr.image_id, df_attr.party))
total_faces = df_faces.columns[7:]

df_fid_list =[]
for f_id in total_faces:
    df_face = df_faces[df_faces[f_id]> 0][['upload_date','sentiment','view_count','uploader']]
    df_face['lat'] = float(loc_dict[f_id].split(',')[0])
    df_face['long'] = float(loc_dict[f_id].split(',')[1])
    df_face['name'] = per_dict[f_id]
    df_face['party'] = party_dict[f_id]
    df_face['constituency'] = constituency_dict[f_id]
    df_fid_list.append(df_face[['lat','long','upload_date','name','party','constituency','sentiment','view_count','uploader']])
df_folium_plot = pd.concat(df_fid_list)
df_folium_plot = df_folium_plot.reset_index()
del df_folium_plot['index']
df_folium_plot.to_csv('plots_data/dashboard_folium.csv',index= False)



''' 
tagplot
'''

import pandas as pd
df_tags = pd.read_csv('UIdata/tags.csv')
tags = [i for i in df_tags.columns[7:] if 'proba_' not in i]
df_tid_list =[]
for t_id in tags:
    df_tag = df_tags[df_tags['proba_'+t_id]>0.98][['upload_date','duration','comment_count','view_count','sentiment','uploader']]
    df_tag['tags'] = t_id
    df_tid_list.append(df_tag)
df_tags_plot = pd.concat(df_tid_list)
df_tags_plot = df_tags_plot.reset_index()
del df_tags_plot['index']
df_tags_plot.to_csv('plots_data/dashboard_tags.csv',index= False)



'''
uploader
'''
df_full_data = pd.read_csv('UIdata/data_compiled_final_may_2023.csv')
df_uploader = df_tags_plot[['uploader','upload_date','duration','comment_count','view_count']]
df_uploader['comment_count'] = df_uploader['comment_count'].fillna(0)
df_uploader.to_csv('plots_data/uploader.csv',index= False)


/var/folders/z7/x6vq9qd56_scz4tkv0sj5lgc0000gn/T/ipykernel_25945/1175041767.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Folium Map

In [204]:
import folium
from folium.plugins import HeatMap
df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')
date_start = '2023-01-01'
date_end = '2024-01-01'
df_folium_sorted = df_folium_plot[(df_folium_plot.upload_date >= date_start) & (df_folium_plot.upload_date <= date_end)]
df_folium_sorted['heat'] = df_folium_sorted['view_count']
df_folium_sorted['heat'] = (df_folium_sorted['heat'] - df_folium_sorted['heat'].min())/(df_folium_sorted['heat'].max() - df_folium_sorted['heat'].min())


df_folium_sorted = df_folium_sorted[['lat','long','heat']]
df_folium_sorted = df_folium_sorted.groupby(['lat','long']).mean().reset_index()
lats_longs_weight = df_folium_sorted.values
map_obj = folium.Map(location = [16.315487, 80.232213], zoom_start = 8,)
HeatMap(lats_longs_weight,min_opacit=1,radius=10).add_to(map_obj)

df_markers = df_folium_plot[['lat','long','name']].drop_duplicates()
for i in range(len(df_markers)):
    folium.CircleMarker([df_markers['lat'].values[i], df_markers['long'].values[i]], 
                popup=df_markers['name'].values[i],radius=3,color ='crimson'
                ).add_to(map_obj)
map_obj



/var/folders/z7/x6vq9qd56_scz4tkv0sj5lgc0000gn/T/ipykernel_25945/2396082504.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/z7/x6vq9qd56_scz4tkv0sj5lgc0000gn/T/ipykernel_25945/2396082504.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')

In [32]:
df_folium_plot

,lat,long,upload_date,name,party,constituency,sentiment,view_count,uploader
0,16.784508,81.076493,2023-02-16,Viswasarayi Kalavathi,YSRCP,Viswasarayi Kalavathi,-0.286784,2800.0,TV9
1,16.784508,81.076493,2022-09-14,Viswasarayi Kalavathi,YSRCP,Viswasarayi Kalavathi,-0.957865,34954.0,Sakshi
2,16.784508,81.076493,2022-12-21,Viswasarayi Kalavathi,YSRCP,Viswasarayi Kalavathi,0.898837,143728.0,TV9
3,16.784508,81.076493,2022-10-22,Viswasarayi Kalavathi,YSRCP,Viswasarayi Kalavathi,0.525788,1720.0,TV5
4,16.784508,81.076493,2022-09-10,Viswasarayi Kalavathi,YSRCP,Viswasarayi Kalavathi,-0.074041,551.0,TV5
...,...,...,...,...,...,...,...,...,...
276731,16.765315,80.926481,2022-08-05,gangula brijendra reddy,YSRCP,gangula brijendra reddy,0.495692,26021.0,Sakshi
276732,16.765315,80.926481,2022-10-26,gangula brijendra reddy,YSRCP,gangula brijendra reddy,-0.018102,2006.0,TV5
276733,16.765315,80.926481,2023-02-16,gangula brijendra reddy,YSRCP,gangula brijendra reddy,0.893092,1891.0,TV9
276734,16.765315,80.926481,2022-09-05,gangula brijendra reddy,YSRCP,gangula brijendra reddy,-0.045191,592.0,TV5


## Bubble chart

In [334]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import numpy as np
import plotly.express as px


date_start = '2022-01-01'
date_end = '2024-01-01'
total_candidates_start = 0
total_candidates_end = 10

df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')
df_bubble = df_folium_plot[['constituency','upload_date','sentiment','view_count','party']]
df_bubble_sorted = df_bubble[(df_bubble.upload_date >= date_start) & (df_bubble.upload_date <= date_end)]
df_bubble_grouped = df_bubble_sorted[['constituency','sentiment','view_count','party']].groupby(['constituency','party']).agg({"sentiment": np.mean, 'view_count': np.sum}).reset_index()
df_bubble_grouped  = df_bubble_grouped.sort_values('view_count', ascending = False)
df_bubble_grouped = df_bubble_grouped[total_candidates_start:total_candidates_end]
df_bubble_grouped['area'] = (df_bubble_grouped['view_count']/1e6)**3

df = px.data.gapminder()
fig = px.scatter(df_bubble_grouped, x="view_count", y="sentiment",
	         size="area", color="party",
                 hover_name="constituency", log_x=True, size_max=50,color_discrete_map= {'YSRCP': '#038d45','TDP': '#ffff00', 'JSP':'#f10003' })
fig.update_layout(
    title='Constituency Graph',
    xaxis=dict(
        title='Views',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Sentiment',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)
fig.show()
fig = px.bar(df_bubble_grouped, x="constituency", y="view_count",color='party', log_y=True, text_auto='.2s',color_discrete_map= {'YSRCP': '#038d45','TDP': '#ffff00', 'JSP':'#f10003' })
fig.update_traces(marker_line_width=1.5, opacity=0.6)
fig.update_layout(
    title='Constituency Graph',
    xaxis=dict(title='Constituency'),
    yaxis=dict(title='Views'))

fig.show()

df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')
df_bubble = df_folium_plot[['constituency','upload_date','sentiment','view_count','party']]
df_bubble_grouped = df_bubble_sorted[['constituency','sentiment','view_count','party']].groupby(['constituency','party']).agg({"sentiment": np.mean, 'view_count': np.sum}).reset_index()
df_bubble_grouped  = df_bubble_grouped.sort_values('view_count', ascending = False)

fig = px.treemap(df_bubble_grouped, path=[px.Constant("Parties"), 'party', 'constituency'], values='view_count',
                  color='party', hover_data=['sentiment'],color_discrete_map= {'YSRCP': '#038d45','TDP': '#ffff00', 'JSP':'#f10003' })
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

## Animation

In [124]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import numpy as np
import plotly.express as px

df_folium_plot = pd.read_csv('plots_data/dashboard_folium.csv')
df_anim = df_folium_plot[['constituency','upload_date','sentiment','view_count','party']]
df_anim['upload_date'] = pd.to_datetime( df_anim.upload_date)
df_anim = df_anim.set_index('upload_date')
df_anim_grouped =  df_anim.groupby([(df_anim.index.year), (df_anim.index.month), 'constituency', 'party']).agg({"sentiment": np.mean, 'view_count': np.sum})
df_anim_grouped = df_anim_grouped.reset_index(names=['year', 'month','constituency','party'])

df_anim_grouped['month'] = df_anim_grouped['month'].apply(str)
df_anim_grouped['year'] = df_anim_grouped['year'].apply(str)
df_anim_grouped['month/year'] =  '01' +'-' +df_anim_grouped['month'] + '-' + df_anim_grouped['year'] 
df_anim_grouped['month/year'] = pd.to_datetime(df_anim_grouped['month/year'])
df_anim_grouped = df_anim_grouped.sort_values('month/year', ascending=True)



df = px.data.gapminder()
px.scatter(df_anim_grouped, x="view_count", y="sentiment", animation_frame="month",animation_group='constituency',
           size="view_count", color="party", hover_name="constituency",
           log_x=True, size_max=60)

/var/folders/z7/x6vq9qd56_scz4tkv0sj5lgc0000gn/T/ipykernel_25945/4091317609.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Tags Plot

In [3]:

# prepare data frames
import plotly.graph_objs as go
from plotly import tools

date_start = '2022-01-01'
date_end = '2024-01-01'
total_tags_start = 0
total_tags_end = 40

df_tags_plot = pd.read_csv('plots_data/dashboard_tags.csv')
df_bar = df_tags_plot
df_bar_sorted = df_bar[(df_bar.upload_date >= date_start) & (df_bar.upload_date <= date_end)]

df_bar_grouped = df_bar_sorted.groupby(['tags']).agg({"sentiment": np.mean, 'view_count': np.sum, 'comment_count':np.sum,'duration':np.sum}).reset_index()
df_bar_grouped  = df_bar_grouped.sort_values('view_count', ascending = False)
df_bar_grouped = df_bar_grouped[total_tags_start:total_tags_end]
df_bar_grouped['duration'] = df_bar_grouped['duration']/(60*60)

tags = df_bar_grouped['tags'].values
views = df_bar_grouped['view_count'].values
duration = df_bar_grouped['duration'].values


y_saving = views
y_net_worth  = duration
x_saving = tags
x_net_worth  = tags



trace0 = go.Bar(
                x=y_saving,
                y=x_saving,
                marker=dict(color='rgba(171, 50, 96, 0.6)',line=dict(color='rgba(171, 50, 96, 1.0)',width=1)),
                name='Views',
                orientation='h',
)
trace1 = go.Scatter(
                x=y_net_worth,
                y=x_net_worth,
                mode='lines+markers',
                line=dict(color='rgb(63, 72, 204)'),
                name='Duration',
)
layout = dict(
                title='Tags Views and Duration',
                yaxis=dict(showticklabels=True,domain=[0, 0.85]),
                yaxis2=dict(showline=True,showticklabels=False,linecolor='rgba(102, 102, 102, 0.8)',linewidth=2,domain=[0, 0.85]),
                xaxis=dict(zeroline=False,showline=False,showticklabels=True,showgrid=True,domain=[0, 0.42]),
                xaxis2=dict(zeroline=False,showline=False,showticklabels=True,showgrid=True,domain=[0.47, 1],side='top',dtick=25),
                legend=dict(x=0.029,y=1.038,font=dict(size=10) ),
                margin=dict(l=200, r=20,t=70,b=70),
                paper_bgcolor='rgb(248, 248, 255)',
                plot_bgcolor='rgb(248, 248, 255)',
)
annotations = []
y_s = np.round(y_saving, decimals=2)
y_nw = np.rint(y_net_worth)
for ydn, yd, xd in zip(y_nw, y_s, x_saving):
    annotations.append(dict(xref='x2', yref='y2', y=xd, x=ydn - 4,text='{:,}'.format(ydn),font=dict(family='Arial', size=12,color='rgb(63, 72, 204)'),showarrow=False))
    annotations.append(dict(xref='x1', yref='y1', y=xd, x=yd + 3,text=str(yd),font=dict(family='Arial', size=12,color='rgb(171, 50, 96)'),showarrow=False))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.update_layout(
    title='Constituency Graph',
    xaxis=dict(title='Views'),
    yaxis=dict(title='Tags'))
fig['layout'].update(layout)
fig.show()



df_tags_plot = pd.read_csv('plots_data/dashboard_tags.csv')
df_bar_grouped = df_tags_plot.groupby(['tags','uploader']).agg({"sentiment": np.mean, 'view_count': np.sum, 'comment_count':np.sum,'duration':np.sum}).reset_index()
df_bar_grouped  = df_bar_grouped.sort_values('view_count', ascending = False)
df_bar_grouped['duration'] = df_bar_grouped['duration']/(60*60)

fig = px.treemap(df_bar_grouped, path=[px.Constant("Media"), 'uploader', 'tags'], values='view_count',
                  color='uploader', hover_data=['duration','sentiment'],color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'ff2a26' , 'TV9':'#0000FF'})
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

/Users/kowshik/projects/personal/alphapolitica_streamlit/streamenv/lib/python3.8/site-packages/plotly/tools.py:460: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



## Media Analytics

In [13]:
import plotly.express as px
from plotly.subplots import make_subplots
                    

import plotly.graph_objs as go
from plotly import tools

date_start = '2022-01-01'
date_end = '2024-01-01'
df_upload_plot = pd.read_csv('UIdata/uploader.csv')

df_bar = df_upload_plot
df_bar_sorted = df_bar[(df_bar.upload_date >= date_start) & (df_bar.upload_date <= date_end)]
df_bar_grouped = df_bar_sorted.groupby(['uploader']).agg({"sentiment": np.mean, 'view_count': np.sum, 'comment_count':np.sum,'duration':np.sum,}).reset_index()
df_bar_grouped['uploads'] = df_bar_sorted.groupby(['uploader']).count().reset_index()['sentiment']
df_bar_grouped  = df_bar_grouped.sort_values('view_count', ascending = False)
df_bar_grouped['duration'] = df_bar_grouped['duration']/(60*60*25)



fig = px.pie(df_bar_grouped, values='view_count', names='uploader',
             title='Views Distribution',color='uploader',
             hover_data=['view_count'],hole=.3,color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'ff2a26' , 'TV9':'#0000FF'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


fig = px.pie(df_bar_grouped, values='comment_count', names='uploader',
             title='comment Distribution',color='uploader',
             hover_data=['comment_count'],hole=.3,color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'ff2a26' , 'TV9':'#0000FF'})

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


fig = px.pie(df_bar_grouped, values='duration', names='uploader',
             title='duration Distribution',color='uploader',
             hover_data=['duration'],hole=.3,color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'ff2a26' , 'TV9':'#0000FF'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


fig = px.pie(df_bar_grouped, values='uploads', names='uploader',
             title='uploads Distribution',color='uploader',
             hover_data=['uploads'],hole=.3,color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'ff2a26' , 'TV9':'#0000FF'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [30]:
import plotly.express as px
df_upload_plot = pd.read_csv('UIdata/uploader.csv')
df_upload_line = df_upload_plot.groupby(['uploader','upload_date']).aggregate({'duration':np.sum,
                                                            'comment_count':np.sum,
                                                            'view_count':np.sum,
                                                            'sentiment':np.mean}).reset_index()
df_upload_line = df_upload_line[df_upload_line.upload_date > '2022-11-01']

df_all_upload_ = []
for upl in ['ABN','TV5','TV9','Sakshi']:
    df_channel = df_upload_line[df_upload_line.uploader == upl].sort_values('upload_date')
    for col in ['duration','comment_count','view_count','sentiment']:
        df_channel[col] = df_channel[col].rolling(10).mean()
    df_all_upload_.append(df_channel)
df_line_chart = pd.concat(df_all_upload_)

fig = px.line(df_line_chart, x='upload_date', y='duration', color='uploader',color_discrete_map= {'ABN': 'yellow','TV5': 'gold', 'Sakshi':'red' , 'TV9':'blue'}, markers=False)
fig.show()

,uploader,upload_date,duration,comment_count,view_count,sentiment
31,ABN,2022-11-11,693095.4,1220.0,3302937.1,-0.019112
32,ABN,2022-11-12,686522.1,1299.2,3394127.1,-0.026047
33,ABN,2022-11-13,656422.8,1265.8,3185816.4,-0.020132
34,ABN,2022-11-14,631641.5,1198.4,3082259.7,-0.019564
35,ABN,2022-11-15,603510.8,1120.8,3012781.6,-0.030632
...,...,...,...,...,...,...
979,Sakshi,2023-04-18,52046.2,1148.1,711271.8,0.003439
980,Sakshi,2023-04-19,47337.8,1235.7,758763.8,0.001905
981,Sakshi,2023-04-20,43911.3,1171.2,699642.4,-0.010019
982,Sakshi,2023-04-21,41123.9,1135.7,657978.5,-0.015359


In [318]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


,tags,uploader,sentiment,view_count,comment_count,duration
243,tag18,TV9,-0.004078,23464837.0,8886.0,221.762222
590,tag97,TV5,-0.037824,22310222.0,15326.0,356.587778
183,tag139,TV9,0.006230,21102917.0,9185.0,226.424722
223,tag148,TV9,0.014320,20356671.0,6493.0,187.078056
579,tag94,TV9,-0.015693,19996053.0,7319.0,185.166111
...,...,...,...,...,...,...
437,tag62,Sakshi,-0.070441,6523267.0,8976.0,102.713611
149,tag131,Sakshi,0.021389,6200152.0,9280.0,110.744722
5,tag1,Sakshi,-0.010062,6179785.0,7915.0,85.843611
289,tag29,Sakshi,0.038034,6105906.0,10276.0,103.733611
